In [6]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Covariance

In [20]:
def exponential_cov(x, y, params):
    return params[0] * np.exp( -0.5 * params[1] * np.subtract.outer(x, y)**2)